### Security Market Line

증권시장선(Security Market Line)은 1985년 Dybvig가 Capital Asset Pricing Model을 기반으로 제시한 모형으로, 시스템 리스크인 자산의 beta와 자산 수익률의 관계로 표현된다

$$E(r_i) - r_f = \alpha_0 + \alpha_1 \frac{E(r_i) - r_f}{E(r_m) - r_f} + \epsilon_i, ~~~~~ \epsilon_i \sim N(0,\sigma_i^2)$$

여기서 $E(r_i)$는 자산 i의 수익률이고, $r_f$는 무위험자산의 수익률, $E(r_m)$은 시장의 수익률이다. 즉, $E(r_m) - r_f$은 무위험시장 대비 시장의 초과수익률이고, $E(r_i) - r_f$는 무위험시장 대비 자산의 초과수익률이다

$\frac{E(r_i) - r_f}{E(r_m) - r_f}$는 자산의 beta라고 불리며, 포트폴리오 보유에 대한 risk premium이라고 해석할 수 있다.

오늘 모형에서는 아래와 같이 설명변수가 하나 더 추가된 SML model을 사용한다

$$E(r_i) - r_f = \alpha_0 + \alpha_1 \frac{E(r_i) - r_f}{E(r_m) - r_f} + \alpha_2 \sigma_i + \epsilon_i, ~~~~~ \epsilon_i \sim N(0,\sigma_i^2)$$

#### 1. Data Import

`feature_matrix.csv`는 French 교수가 조사한 미국 100개 산업별 portfolio 데이터 중 2010년부터 2023년 8월까지의 일별 수익률을 기반으로 제작된 데이터이다. 아래와 같은 column을 가지고 있다.

- return : 포트폴리오의 초과 수익률
- beta : 각 포트폴리오의 추정된 리스크 프리미엄(beta)
- volatility : 각 포트폴리오의 추정된 변동성(sigma)

아래의 라이브러리를 이용하여 data를 import하라

In [1]:
import pandas as pd
import numpy as np

In [2]:
matrix = pd.read_csv('feature_matrix.csv', index_col = 0)

#### 2. Linear Regression

아래의 library를 사용하여 OLS Model로 적합화하라. train test split은 0.3으로 지정하고 `random_state = 42`로 지정한다. test 데이터에 대해 추정된 MSE와 R2 score는 얼마인가?

In [3]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

X = matrix[['beta','volatility']]
X = sm.add_constant(X)
y = matrix['return']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

model = sm.OLS(y_train, X_train)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 return   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     3.370
Date:                Thu, 09 Nov 2023   Prob (F-statistic):             0.0403
Time:                        16:08:57   Log-Likelihood:                 218.62
No. Observations:                  70   AIC:                            -431.2
Df Residuals:                      67   BIC:                            -424.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0539      0.009      6.190      0.0

In [4]:
from sklearn.metrics import mean_squared_error, r2_score
y_pred = result.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R squared score: {r2}")

Mean Squared Error: 0.00018299014352184142
R squared score: 0.052096134258785676


#### 3. Ridge Regression

아래의 library를 사용하여 Ridge Regression으로 적합화하라. Hyperparameter `alpha = 0.001`로 지정한다. test 데이터에 대해 추정된 MSE와 R2 score는 얼마인가?

In [5]:
from sklearn.linear_model import Ridge

X = matrix[['beta','volatility']]
y = matrix['return']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

linear = Ridge(alpha = 0.001)
linear.fit(X_train, y_train)

Ridge(alpha=0.001)

In [6]:
print(linear.coef_)

[ 0.00835332 -0.29978781]


In [7]:
y_train_linear = linear.predict(X_train)
mse_linear = mean_squared_error(y_train, y_train_linear)
r2_linear = r2_score(y_train, y_train_linear)

print(f"Ridge Regression with Train Data - MSE: {mse_linear}, R-squared: {r2_linear}")

Ridge Regression with Train Data - MSE: 0.00012022951975268241, R-squared: 0.037179632052277034


In [8]:
y_pred_linear = linear.predict(X_test)
mse_linear = mean_squared_error(y_test, y_pred_linear)
r2_linear = r2_score(y_test, y_pred_linear)

print(f"Ridge Regression - MSE: {mse_linear}, R-squared: {r2_linear}")

Ridge Regression - MSE: 0.0001907898166641984, R-squared: 0.011693191341394904


#### 4. Support Vector Regression

아래의 library를 사용하여 Support Vector Regression으로 적합화하라. Hyperparameter `C = 10`로 지정한다. kernel은 linear kernel을 사용하도록 한다. test 데이터에 대해 추정된 MSE와 R2 score는 얼마인가?

In [20]:
from sklearn.svm import SVR

X = matrix[['beta','volatility']]
y = matrix['return']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

model = SVR(kernel = 'linear', C = 0.01)
model.fit(X_train, y_train)

SVR(C=0.01, kernel='linear')

In [21]:
model.coef_

array([[0., 0.]])

In [19]:
y_train_linear = model.predict(X_train)
mse_linear = mean_squared_error(y_train, y_train_linear)
r2_linear = r2_score(y_train, y_train_linear)

print(f"Support Vector Regression with Train Data - MSE: {mse_linear}, R-squared: {r2_linear}")

Support Vector Regression with Train Data - MSE: 0.00012798424559097983, R-squared: -0.024921655554310584


In [12]:
y_pred_linear = model.predict(X_test)
mse_linear = mean_squared_error(y_test, y_pred_linear)
r2_linear = r2_score(y_test, y_pred_linear)

print(f"Support Vector Regression - MSE: {mse_linear}, R-squared: {r2_linear}")

Support Vector Regression - MSE: 0.00019489540437004214, R-squared: -0.009574087773255302


#### 5. Regression Tree

아래의 library를 사용하여 DecisionTree Regressor로 적합화하라. Hyperparameter `creterion = 'squared_error', max_depth = 4, random_state = 42`로 지정한다. test 데이터에 대해 추정된 MSE와 R2 score는 얼마인가?

In [13]:
from sklearn.tree import DecisionTreeRegressor

X = matrix[['beta','volatility']]
y = matrix['return']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

tree = DecisionTreeRegressor(criterion = 'squared_error',
                             max_depth = 4,
                             random_state = 42)

tree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=4, random_state=42)

In [14]:
tree.feature_importances_

array([0.35966764, 0.64033236])

In [15]:
y_train_linear = tree.predict(X_train)
mse_linear = mean_squared_error(y_train, y_train_linear)
r2_linear = r2_score(y_train, y_train_linear)

print(f"Tree Regression with Train Data - MSE: {mse_linear}, R-squared: {r2_linear}")

Tree Regression with Train Data - MSE: 7.992443505323941e-05, R-squared: 0.35995025078475384


In [16]:
y_pred = tree.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R squared score: {r2}")

Mean Squared Error: 0.0002021367743690289
R squared score: -0.047084974880039265


#### 6. 모형 추정 결과

각 모형별 추정 결과를 비교하라. 어떤 모형을 사용하기에 가장 적합한가? 금융에서 머신러닝 프로젝트가 종종 실패하는 이유를 추론할 수 있는가?